In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from tensorboardX import SummaryWriter
from torch.optim.lr_scheduler import LambdaLR, StepLR

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
class Config:

    batch_size = 64
    epochs = 300
    lr = 1e-3

    channel = 3

    height = 32
    width = 32

    data_root = '../dataset/cifar10'

    dropout_rate = 0.1
    attn_dropout = 0

    patch_size = 4
    num_patches = int((height * width) / (patch_size ** 2))

    layers = 12
    embedding_d = 768
    mlp_size = 1024
    heads = 8

    num_classes = 10

    log_f = 100

In [19]:
transform = transforms.Compose([
  transforms.Resize(224),
  transforms.CenterCrop(224),
  transforms.ToTensor(),
  transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

In [20]:
trainset = torchvision.datasets.CIFAR10(root=Config.data_root, train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=Config.batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=Config.data_root, train=False,
                                       download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=Config.batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
!pip install timm

In [23]:
import timm

model = timm.create_model("vit_base_patch16_224", pretrained=True)
model.head = nn.Linear(model.head.in_features, 10)

In [24]:
# model = Model().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=Config.lr)
criterion = nn.CrossEntropyLoss()

In [25]:
scheduler = StepLR(optimizer, 5)

In [26]:
writer = SummaryWriter()

In [27]:
def test(epoch):

    model.eval()

    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():

        for batch_idx, (inputs, targets) in enumerate(tqdm(testloader)):

            if Config.batch_size != len(inputs):
                inputs = inputs.repeat(Config.batch_size // inputs.size(0) + 1, 1, 1, 1)[:Config.batch_size]
                targets = targets.repeat(Config.batch_size // targets.size(0) + 1)[:Config.batch_size]

            inputs, targets = inputs.to(device), targets.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total

    print(f'Epoch {epoch} val loss: {test_loss:.5f}, test acc: {(acc):.5f}')

    return test_loss, acc

In [28]:
def train(epoch):

    model.train()

    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(tqdm(trainloader)):

        if Config.batch_size != len(inputs):
            inputs = inputs.repeat(Config.batch_size // inputs.size(0) + 1, 1, 1, 1)[:Config.batch_size]
            targets = targets.repeat(Config.batch_size // targets.size(0) + 1)[:Config.batch_size]

        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        acc = 100.*correct/total

        if batch_idx % Config.log_f == 0:
            print(f'Epoch {epoch}, batch index {batch_idx} || train loss: {train_loss/(batch_idx+1)}, train acc: {acc}')

    return train_loss/(batch_idx+1), acc

In [29]:
model = model.to(device)

In [30]:
for epoch in range(Config.epochs):

    train_loss, train_acc = train(epoch)
    test_loss, test_acc = test(epoch)

    scheduler.step()

    writer.add_scalar('train/loss', train_loss, epoch)
    writer.add_scalar('train/acc', train_acc, epoch)
    writer.add_scalar('test/loss', test_loss, epoch)
    writer.add_scalar('test/acc', test_acc, epoch)

  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 0, batch index 0 || train loss: 2.1957013607025146, train acc: 25.0
Epoch 0, batch index 100 || train loss: 2.356283929088328, train acc: 19.198638613861387
Epoch 0, batch index 200 || train loss: 2.1450099440949475, train acc: 23.34421641791045
Epoch 0, batch index 300 || train loss: 2.0265084465476764, train acc: 26.28737541528239
Epoch 0, batch index 400 || train loss: 1.9504247966252657, train acc: 28.56920199501247
Epoch 0, batch index 500 || train loss: 1.8881200804205949, train acc: 30.713572854291417
Epoch 0, batch index 600 || train loss: 1.8372360134680141, train acc: 32.43032445923461
Epoch 0, batch index 700 || train loss: 1.7999802239441158, train acc: 33.73083095577746


  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 0 val loss: 451.13953, test acc: 16.47094


  0%|          | 0/782 [00:00<?, ?it/s]

...

  0%|          | 0/157 [00:00<?, ?it/s]

Epoch 9 val loss: 465.68100, test acc: 21.93471


  0%|          | 0/782 [00:00<?, ?it/s]

Epoch 10, batch index 0 || train loss: 1.0078420639038086, train acc: 60.9375
Epoch 10, batch index 100 || train loss: 1.0800429759639325, train acc: 60.9375


KeyboardInterrupt: 